In [1]:
import numpy as np
import re
import pandas as pd
import geopandas as gpd
import gdown
import ee
import google
import os
import geemap

from tobler.util import h3fy
from ee_jupyter.ipyleaflet import Map

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1Adeu5BX3Wosqc0q9MpigKESYiyn53Y8qVpSLbl-JQj7SgHfQiJpjr3Ho81U

Successfully saved authorization token.


In [ ]:
CGAZ_ADM1 = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM1')

# Filter Comprehensive Global Administrative Zones ADM1 for AK
ak = CGAZ_ADM1.filter(ee.Filter.eq('shapeISO', 'US-AK'))
# Convert the feature collection to a geometry.
akGeometry = ak.geometry()
# Get the bounding box of the geometry.
akBounds = akGeometry.bounds()

# same process for Juneau
juneau = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('COUNTYFP', '110')).filter(ee.Filter.eq('STATEFP', '02'))
juneauGeometry = juneau.geometry()
juneauBounds = juneauGeometry.bounds()

In [216]:
temp = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate('2010-01-01', '2019-12-31').select('temperature_2m').map(lambda image: image.clip(juneauGeometry))

In [80]:
temp.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'temperature_2m',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [3601, 1801],
   'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}],
 'version': 1680765300915096,
 'id': 'ECMWF/ERA5_LAND/HOURLY/20100101T00',
 'properties': {'system:time_start': 1262304000000,
  'hour': 0,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1262307600000,
  'system:asset_size': 395595778,
  'system:index': '20100101T00'}}

In [228]:
def temporalReduction(temp):
    # create a list for every day-hour combo
    dates = []
    for d in range(0, 365):
        for h in range(0, 24):
            dates.append((d, h))
    
    # empty list to store the 10 year average images
    collection = []
    # calculate averages for every day-hour combo
    for day, hour in dates:
        average = temp.filter(ee.Filter.dayOfYear(day, day)).filter(ee.Filter.eq('hour', hour)).mean()
        collection.append(average)
        
    return ee.ImageCollection(collection)

temp_10year_average = temporalReduction(temp)
type(temp_10year_average), type(temp_10year_average.first())

<class 'ee.imagecollection.ImageCollection'>


ee.image.Image

In [218]:
# one_hour = temp.filter(ee.Filter.dayOfYear(2, 2)).filter(ee.Filter.eq('hour', 2)).mean()

In [229]:
# map1 = Map(center=[134.4201, 58.3005], zoom=10)

# map1.addLayer(one_hour, {}) #, {'min': 230, 'max': 250, 'palette': ['grey', 'blue']})

# map1

In [139]:
buildings = ee.FeatureCollection('projects/sat-io/open-datasets/ORNL/USA-STRUCTURES/USA_ST_AK')

image = ee.Image(temp_10year_average.first())

buildingsRaster = ee.Image().byte().paint(featureCollection = buildings, color = 1)
# Get information about the ERA5-Land projection.
imageProjection = image.projection()

# Force the next reprojection to aggregate instead of resampling.
# The reduce resolution returns the fraction of the ERA-5 Land pixel
# that's covered by 30 meter forest pixels.  Convert to area
# after the reduceResolution() call.
# Request the data at the scale and projection of the ERA-5 Land image.

buildingsRaster = buildingsRaster.setDefaultProjection('EPSG:4326')

imageBuildings = buildingsRaster.gt(0).reduceResolution(reducer = ee.Reducer.mean(),maxPixels = 1024) \
    .multiply(ee.Image.pixelArea()).reproject(crs = imageProjection)

# Import ERA-5 Land, any date will do
dataset = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filter(ee.Filter.date('2020-07-01', '2020-07-02'));

# Get single image composite over AK, have to reproject because
# mosiacs do not have default projection
image = dataset.max().clip(juneauGeometry).select('temperature_2m').reproject('EPSG:4326', None, 11132)

# Rasterize buildings
buildingsRaster = ee.Image().byte().paint(featureCollection = buildings, color = 1)

# Get coverage of buildings per ERA-5 Pixel

#  Force the next reprojection to aggregate instead of resampling.
#  The reduce resolution returns the fraction of the ERA-5 Land pixel
#  that's covered by 100 meter building pixels.  Convert to area
#  after the reduceResolution() call.
#  Request the data at the scale and projection of the ERA-5 Land image.
imageBuildings = buildingsRaster.unmask(0).setDefaultProjection('EPSG:4326', None, 100).reduceResolution(
      reducer = ee.Reducer.mean(), maxPixels = 12393).multiply(ee.Image.pixelArea()).reproject('EPSG:4326', None, 11132)


In [140]:
type(imageBuildings)

ee.image.Image

In [141]:
centroids = imageBuildings.gt(0).sample(
  region = akGeometry,
  geometries =  True);

Notes from convo with Erin:

join centroids (small blocks) or imageBuildings mask (larger blocks)

updateMask() function mapped on 10year average, then unmask 0, then spatially reduce

In [232]:
def maskTemp(image):
    return image.updateMask(imageBuildings).unmask(0)

temp_10year_average_masked = temp_10year_average.map(maskTemp)

In [237]:
type(temp_10year_average_masked), type(temp_10year_average_masked.first())

(ee.imagecollection.ImageCollection, ee.image.Image)

In [235]:
# the below will get memory errors which seems wrong...

# map1 = Map(center=[134.4201, 58.3005], zoom=4)

# map1.addLayer(temp_10year_average_masked.first(), {}) 

# map1

In [149]:
blocks = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Blocks2020/Blocks2020.shp')
blocks = blocks[['NAME', 'STATE', 'BOROUGH', 'TRACT', 'BLOCK', 'FIPS', 'TOTALPOP', 'HOUSEUNITS', 'geometry']]
juneauBlocks = blocks[(blocks['BOROUGH'] == '110') & (blocks['TOTALPOP'] > 0)]
juneauBlocksFC = geemap.geopandas_to_ee(juneauBlocks)

In [150]:
juneauBlocksFC.size().getInfo()

380

In [151]:
areas = juneauBlocks['geometry'].area
areas.min(), areas.median(), areas.max()

(1055.8870653038284, 27503.41284678245, 2220401169.367476)

In [238]:
# spatial reduction according to juneauBlocksFC

def reduce_to_region(image):
    reduced_image = image.reduceRegions(collection=juneauBlocksFC, reducer=ee.Reducer.mean(), scale=1000)
    #reduced_image = reduced_image.set('date', image.get('julian_date'))
    return reduced_image

export = temp_10year_average_masked.map(reduce_to_region)

In [239]:
type(export), type(export.first())

(ee.imagecollection.ImageCollection, ee.image.Image)

In [240]:
export1 = export.first()

In [242]:
task = ee.batch.Export.table.toDrive(**{
  'collection': export1,
  'description': 'juneau block export for one block',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()

In [245]:
task = ee.batch.Export.table.toDrive(**{
  'collection': export,
  'description': 'juneau block export try 1',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()